In [ ]:
# use widgets as matplotlib backend
%matplotlib widget

# imports
from matplotlib import pyplot as plt
import os
import numpy
import os, sys
from time import time
from tqdm.auto import tqdm
from collections import defaultdict
import numpy as np
import matplotlib as mpl

# import the file object for opening kongsberg files
# Note: function and library naming to be discussed
from themachinethatgoesping.echosounders import kongsbergall
from themachinethatgoesping.echosounders import index_functions
from themachinethatgoesping.pingprocessing import filter_pings

#simplify creating figures
mpl.rcParams['figure.dpi'] = 100
close_plots = True

def create_figure(name, return_ax=True):
    if close_plots: plt.close(name)
    fig = plt.figure(name)
    fig.suptitle = name

    if return_ax:
        return fig, fig.subplots()
    return fig

In [ ]:
# list of folders with kongsberg .all or .wcd files (subfolders will be scanned as well)

folder = "../../unittest_data/"

# list raw data files
files = index_functions.find_files(folder, [".all","wcd"])
cacheFilePaths = index_functions.get_cache_file_paths(file_paths=files)
files.sort()
            
files.sort()
file_name = files[0]
print("files:      ", len(files))

## Open all files
The function 
Notes: 
1. kongsbergall.KongsbergAllFileHandler_mapped(files) scanns and indexes all files and provides access to all files like a combined file stream
2. If a .all and a .wcd files with the same name (one .all and one .wcd) are added, they will be matched to a single file
3. It is not possible to add two .all or two .wcd with the same name, even if they are within different folders
4. Note: if the files are not sorted in time, the datagram packages will not be sorted by time either, however it isi simple to sort the pings at a later stage

In [ ]:
# Index all files and initialize the file interfaces
# fm will be the accessor
fm = kongsbergall.KongsbergAllFileHandler_mapped(files)

#print some information about the files that where indexed
print(fm)

## How to access ping data
1. Pings describe the main highlevel interface of Ping
2. They are easy to access using fm.get_pings
3. Pings combine information from the interfaces (e.g. navigation or configuration) with raw datagrams that where produced for each ping
4. Pings implement simple functions to access difficult data (e.g get_sv) However, this is still work in progress
5. Pings also provide access to the raw data, this will be used for plotting later

In [ ]:
# Extract the 11th single ping for test
ping = fm.get_pings()[111]

# print the georeferenced position and attitude of the transducer
print(ping.get_geolocation())

In [ ]:
# filter pings such that the for example must include water column and bottom detection data
pings = filter_pings.by_features(fm.get_pings(),['watercolumn','bottom'])
ping = pings[5]

In [ ]:
# get access to the raw data of the specific ping
raw = ping.file_data

# print / access datagrams associated with the ping
print(raw.datagrams())

In [ ]:
#print the RawRangeAndAngle datagram associated with the ping
print(raw.datagrams("WatercolumnDatagram")[0])

In [ ]:
# You may know that water column datagrams in the wcd/all files are split over multiple packtes.
# We therefore implemented a function that combines the split datagrams to a single one
wcd = raw.read_merged_watercolumndatagram()

print(wcd)

In [ ]:
# read_merged_watercolumndatagram() holds a variable that is called skip_data
# if the true, the sample data will be skipped
# in this case you cannot access sample data, but looping through the data is accelerated 
# This is usefull to extract e.g statistical information in a first loop
# Speedup is not very impressive though (typically factor 2) because of the complicated datagram structure of .wcd files

# check if there where different tvgs applied in the datasets
tvgs = defaultdict(int)
for i,p in enumerate(tqdm(pings)):
    try:    
        w = p.file_data.read_merged_watercolumndatagram(skip_data = True)
        tvgs[w.get_tvg_function_applied()] += 1
              
    except IndexError as e:
        print("error:",i,e,"|",type(e))
    except ValueError as e:
        print("error:",i,e,"|",type(e))
    except RuntimeError as e:
        print("error:",i,e,"|",type(e))

for k,v in tvgs.items():
    print(f"TVG {k}: {v} pings")